In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import load_model

# Load your pre-processed dataset
df = pd.read_csv(r'C:\Users\M.ANTONY ROJES\Downloads\Infosys\data\feature_engineered\player_features_model_all_imputed.csv')

# Define features and target (same as Week 5)
features = ['passes_attempted', 'expected_goals', 'goals', 'assists', 'injury_count', 'total_days_out', 'avg_market_value']
target = 'avg_market_value'

# Create time-series sequences and split data (same as Week 5)
n_steps = 3
def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

# Scale data and create sequences
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features])
X, y = create_sequences(scaled_data, n_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Load the trained LSTM model (assuming you saved it)
# lstm_model = load_model('lstm_model.h5')
# Note: Since the model was not saved, you will use the one created in the previous session.
# If you restart your kernel, you will need to re-run the Week 5 code to recreate the model object.

In [ ]:
# Generate predictions from the LSTM model on the training set
lstm_train_preds = model.predict(X_train).flatten()

# Reshape the original training data for XGBoost (flatten the time steps)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)

# Add the LSTM predictions as a new column to the training data
X_train_ensemble = np.hstack((X_train_reshaped, lstm_train_preds.reshape(-1, 1)))

print("Original training data shape for XGBoost:", X_train_reshaped.shape)
print("Ensemble training data shape with LSTM predictions:", X_train_ensemble.shape)

In [3]:
import pandas as pd
import numpy as np
import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# --- WEEK 5 CODE: DATA PREPARATION AND MODEL TRAINING ---
# Load the consolidated dataset
# Note: You need to replace the local path with the correct file name if it's in the same directory
df = pd.read_csv(r'C:\Users\M.ANTONY ROJES\Downloads\Infosys\data\feature_engineered\player_features_model_all_imputed.csv')

# Define features and target variable
features = ['passes_attempted', 'expected_goals', 'goals', 'assists', 'injury_count', 'total_days_out', 'avg_market_value']
target = 'avg_market_value'

# Scale the data using MinMaxScaler
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features])

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data) - 1:
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 3
X, y = create_sequences(scaled_data, n_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

# Re-build and re-train the model
model = Sequential()
model.add(LSTM(50, activation='relu', input_shape=(n_steps, X.shape[2])))
model.add(Dense(1))
model.compile(optimizer='adam', loss='mean_squared_error')
model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)
print("LSTM model has been successfully rebuilt and trained.")

# --- WEEK 6 CODE: ENSEMBLE MODELING ---
# Generate predictions from the LSTM model on the training and test sets
lstm_train_preds = model.predict(X_train).flatten()
lstm_test_preds = model.predict(X_test).flatten()

# Reshape the original data for XGBoost (flatten the time steps)
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

# Add the LSTM predictions as a new column to the training and test data
X_train_ensemble = np.hstack((X_train_reshaped, lstm_train_preds.reshape(-1, 1)))
X_test_ensemble = np.hstack((X_test_reshaped, lstm_test_preds.reshape(-1, 1)))

print("\nEnsemble training data shape with LSTM predictions:", X_train_ensemble.shape)
print("Ensemble test data shape with LSTM predictions:", X_test_ensemble.shape)

# Build and train the XGBoost model
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)

xgboost_model.fit(X_train_ensemble, y_train)
print("\nXGBoost model trained successfully.")

# Make final predictions with the XGBoost ensemble model
final_preds_scaled = xgboost_model.predict(X_test_ensemble)

# Calculate RMSE for the standalone LSTM model (on the test set)
lstm_rmse = np.sqrt(mean_squared_error(y_test, lstm_test_preds))
print(f"LSTM Model RMSE on Test Set: {lstm_rmse:.4f}")

# Calculate RMSE for the final ensemble model
ensemble_rmse = np.sqrt(mean_squared_error(y_test, final_preds_scaled))
print(f"XGBoost Ensemble Model RMSE on Test Set: {ensemble_rmse:.4f}")

# Compare the results
if ensemble_rmse < lstm_rmse:
    print("\n✅ The ensemble model performed better than the standalone LSTM model!")
else:
    print("\n⚠️ The ensemble model's performance was not an improvement over the standalone LSTM model. You may need to tune hyperparameters or try a different stacking approach.")

c:\Users\M.ANTONY ROJES\Downloads\Infosys\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


LSTM model has been successfully rebuilt and trained.
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 

Ensemble training data shape with LSTM predictions: (916, 22)
Ensemble test data shape with LSTM predictions: (229, 22)

XGBoost model trained successfully.
LSTM Model RMSE on Test Set: 0.1092
XGBoost Ensemble Model RMSE on Test Set: 0.1200

⚠️ The ensemble model's performance was not an improvement over the standalone LSTM model. You may need to tune hyperparameters or try a different stacking approach.


In [4]:
# ================================
# 📌 Week 6: Ensemble Modeling
# ================================

import pandas as pd
import numpy as np
import tensorflow as tf
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

# 1️⃣ Load Data
df = pd.read_csv(r"C:\Users\M.ANTONY ROJES\Downloads\Infosys\data\feature_engineered\player_features_model_all_imputed.csv")

features = ['passes_attempted', 'expected_goals', 'goals', 'assists',
            'injury_count', 'total_days_out', 'avg_market_value']
target = 'avg_market_value'

# 2️⃣ Preprocess Data
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(df[features])

def create_sequences(data, n_steps):
    X, y = [], []
    for i in range(len(data)):
        end_ix = i + n_steps
        if end_ix > len(data)-1:
            break
        seq_x, seq_y = data[i:end_ix, :], data[end_ix, -1]
        X.append(seq_x)
        y.append(seq_y)
    return np.array(X), np.array(y)

n_steps = 3
X, y = create_sequences(scaled_data, n_steps)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print("✅ Data prepared")
print("X_train shape:", X_train.shape, "y_train shape:", y_train.shape)

# 3️⃣ Build & Train LSTM
model = Sequential([
    LSTM(50, activation='relu', input_shape=(n_steps, X.shape[2])),
    Dense(1)
])
model.compile(optimizer='adam', loss='mean_squared_error')
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_test, y_test), verbose=0)

print("✅ LSTM model trained")

# 4️⃣ LSTM Predictions
lstm_train_preds = model.predict(X_train).flatten()
lstm_test_preds = model.predict(X_test).flatten()

# 5️⃣ Prepare Ensemble Input for XGBoost
X_train_reshaped = X_train.reshape(X_train.shape[0], -1)
X_test_reshaped = X_test.reshape(X_test.shape[0], -1)

X_train_ensemble = np.hstack((X_train_reshaped, lstm_train_preds.reshape(-1,1)))
X_test_ensemble = np.hstack((X_test_reshaped, lstm_test_preds.reshape(-1,1)))

print("Ensemble Train Shape:", X_train_ensemble.shape)
print("Ensemble Test Shape:", X_test_ensemble.shape)

# 6️⃣ Train XGBoost
xgboost_model = xgb.XGBRegressor(
    objective='reg:squarederror',
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    random_state=42
)
xgboost_model.fit(X_train_ensemble, y_train)
print("✅ XGBoost model trained")

# 7️⃣ Evaluate Models
final_preds_scaled = xgboost_model.predict(X_test_ensemble)

lstm_rmse = np.sqrt(mean_squared_error(y_test, lstm_test_preds))
ensemble_rmse = np.sqrt(mean_squared_error(y_test, final_preds_scaled))

print(f"\n📊 LSTM RMSE on Test Set: {lstm_rmse:.4f}")
print(f"📊 Ensemble RMSE on Test Set: {ensemble_rmse:.4f}")

if ensemble_rmse < lstm_rmse:
    print("✅ Ensemble improved over LSTM!")
else:
    print("⚠️ Ensemble did not improve, consider tuning hyperparameters.")


✅ Data prepared
X_train shape: (916, 3, 7) y_train shape: (916,)


c:\Users\M.ANTONY ROJES\Downloads\Infosys\.venv\lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


✅ LSTM model trained
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
Ensemble Train Shape: (916, 22)
Ensemble Test Shape: (229, 22)
✅ XGBoost model trained

📊 LSTM RMSE on Test Set: 0.1093
📊 Ensemble RMSE on Test Set: 0.1146
⚠️ Ensemble did not improve, consider tuning hyperparameters.


In [ ]:
# --- SAVE WEEK 6 MODELS & RESULTS ---

# Save rebuilt LSTM model
model.save("week6_lstm_model.h5")
print("✅ Rebuilt LSTM model saved as week6_lstm_model.h5")

# Save trained XGBoost ensemble model
import joblib
joblib.dump(xgboost_model, "week6_xgboost_model.pkl")
print("✅ XGBoost model saved as week6_xgboost_model.pkl")

# Align player names with y_test
df = pd.read_csv(r"C:\Users\M.ANTONY ROJES\Downloads\Infosys\data\feature_engineered\player_features_model_all_imputed.csv")
n_steps = 3
player_names = df['player'][len(df) - len(y_test):].reset_index(drop=True)
# Save predictions and RMSE results with player names
results_df = pd.DataFrame({
    "player": player_names,
    "y_test": y_test,
    "lstm_preds": lstm_test_preds,
    "ensemble_preds": final_preds_scaled
})
results_df.to_csv("week6_predictions.csv", index=False)
print("✅ Predictions saved as week6_predictions.csv with player names")

# Save RMSE results
with open("week6_results.txt", "w") as f:
    f.write(f"LSTM RMSE (Test Set): {lstm_rmse:.4f}\n")
    f.write(f"Ensemble RMSE (Test Set): {ensemble_rmse:.4f}\n")
print("✅ RMSE results saved as week6_results.txt")

In [5]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def evaluate_model(y_true, y_pred, model_name="Model"):
    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    mae = mean_absolute_error(y_true, y_pred)
    r2 = r2_score(y_true, y_pred)
    print(f"\n[{model_name} Evaluation]")
    print(f"RMSE: {rmse:.4f}")
    print(f"MAE : {mae:.4f}")
    print(f"R²  : {r2:.4f}")
    return rmse, mae, r2

# Example usage with Week 6 predictions:
evaluate_model(y_test, lstm_test_preds, "LSTM")
evaluate_model(y_test, final_preds_scaled, "Ensemble (LSTM+XGBoost)")



[LSTM Evaluation]
RMSE: 0.1095
MAE : 0.0620
R²  : -0.0287

[Ensemble (LSTM+XGBoost) Evaluation]
RMSE: 0.1118
MAE : 0.0603
R²  : -0.0728


(np.float64(0.11180233035438548), 0.060325400326593094, -0.07277141394520403)